In [18]:
import tweepy
import os

data_dir = "/Users/ianmagnusson/DisasterProject/data/CrisisMMD_v1.0/annotations/"

consumer_key = "PQSsqSxGft69D53XGknXya8cL"
consumer_secret = "dlh6EFLADanVNvdl9Q9Lz9Iv3yQQyHY9vxKTS9BN2skflGd45d"
access_token = "1187382763432771584-rwZbZc9lcOcaVdgxcIgqqCPqoQMo3I"
access_token_secret = "nvnIFiZD9L1u6tZwntl24BtsLhKmkBntJ4dpko82fYKg8"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


with open(data_dir + "hurricane_harvey_final_data.tsv") as fin:
    raw = fin.readlines()

raw = raw[1:]  
user_tweet_counts = {}
not_found = 0
not_auth = 0
user_suspended = 0


In [20]:
for line in raw[500:1000]:
    tweetid = int(line.split()[0][1:-1])
    try:
        user_id = api.get_status(tweetid).author.id
        user_tweet_counts[user_id] = user_tweet_counts.get(user_id, 0) + 1
    except tweepy.TweepError as e:
        if e.api_code == 144:
            not_found += 1
            continue
        elif e.api_code == 179:
            not_auth += 1
            continue
        elif e.api_code == 63:
            user_suspended += 1
            continue
        else:
            print(e)
            break

[{'message': 'Rate limit exceeded', 'code': 88}]


In [21]:

not_found
not_auth
user_suspended

user_tweet_counts


{756551449149046785: 1,
 1682527040: 4,
 2890175659: 1,
 521460388: 1,
 1379720960: 1,
 868856461: 1,
 4724991865: 1,
 1932816230: 2,
 241642996: 1,
 189656687: 1,
 20193551: 1,
 1901928318: 1,
 14404094: 1,
 46001628: 1,
 885727267: 1,
 3000127167: 1,
 2416788982: 2,
 1957768417: 1,
 303438200: 1,
 934399189: 1,
 15937367: 1,
 841861508942450688: 1,
 719690726641164288: 1,
 1013343390: 1,
 713180918178058240: 1,
 183269991: 1,
 2395374864: 1,
 632129599: 1,
 856767565653303296: 1,
 17059735: 1,
 211002286: 1,
 765400544772956160: 1,
 35778794: 1,
 108653141: 1,
 15922073: 1,
 2649925794: 1,
 30591148: 1,
 188712627: 1,
 2908313571: 1,
 27961495: 3,
 21028056: 1,
 19098792: 1,
 1059240924: 1,
 1859495642: 1,
 789581786540113920: 1,
 36229347: 1,
 27583598: 2,
 800914339020632064: 1,
 768898960488402944: 1,
 414946220: 1,
 622539120: 4,
 3098402048: 1,
 2551069075: 1,
 1395158480: 1,
 851467828683091968: 1,
 792651643: 1,
 2435036751: 1,
 4116123674: 1,
 846851726443495425: 1,
 16935395

In [56]:
import json
user_tweet_counts = {}
with open("/Users/ianmagnusson/DisasterProject/data/CrisisMMD_v1.0/json/srilanka_floods_final_data.json") as fin:
    for line in fin:
        json_data = json.loads(line)
        user_id = json_data['user']['id']
        user_tweet_counts[user_id] = user_tweet_counts.get(user_id, 0) + 1
        
    

In [57]:
freq = {}
for val in user_tweet_counts.values():
    freq[val] = freq.get(val, 0) + 1

In [58]:
freq


{1: 629, 2: 51, 3: 18, 7: 1, 8: 1, 4: 3, 5: 4}

In [89]:
import json
userKeys = {}
with open("/Users/ianmagnusson/DisasterProject/data/CrisisMMD_v1.0/json/hurricane_harvey_final_data.json") as fin:
    for line in fin.readlines()[:100]:
        json_data = json.loads(line)
        tweet_id = json_data['id']
        user_id = json_data['user']['id']
        userKeys[user_id] = userKeys.get(user_id, []) + [tweet_id]
        
    

In [103]:
import datetime

not_found = 0
not_auth = 0
user_suspended = 0
prob_401 = 0

startDate = datetime.datetime(2015, 5, 1, 0, 0, 0)
endDate =   datetime.datetime(2020, 12, 31, 0, 0, 0)
allUsers = len(userKeys)
totalTweetsFound = 0
noneUsers = 0
for user in userKeys:
   tweetTally = 0
   tweets = []
   maxDate = userKeys[user][-1] # Set the last value in dict as max date
   try:
       tmpTweets = api.user_timeline(user, max_id = 1047132293000000000)
   except tweepy.TweepError as e:
        if e.api_code == 144:
            not_found += 1
            continue
        elif e.api_code == 179:
            not_auth += 1
            continue
        elif e.api_code == 63:
            user_suspended += 1
            continue
            
        elif e.reason == 'Not authorized.':
            prob_401 += 1
            continue
        else:
            raise e
   for tweet in tmpTweets:
       if tweet.created_at <= endDate and tweet.created_at >= startDate:
           tweets.append(tweet)
           tweetTally += 1
   print('User: ', user, '. \# tweets found Sept-Dec 2017: ', tweetTally)
   totalTweetsFound += tweetTally
   if tweetTally == 0:
       noneUsers += 1
print('\# tweets found total: ', totalTweetsFound, '\# Users: ', allUsers, '\# no tweet users: ', noneUsers)

User:  1682527040 . \# tweets found Sept-Dec 2017:  0
User:  3407291422 . \# tweets found Sept-Dec 2017:  14
User:  2890175659 . \# tweets found Sept-Dec 2017:  20
User:  521460388 . \# tweets found Sept-Dec 2017:  0
User:  1379720960 . \# tweets found Sept-Dec 2017:  7
User:  561449155 . \# tweets found Sept-Dec 2017:  16
User:  868856461 . \# tweets found Sept-Dec 2017:  0
User:  4724991865 . \# tweets found Sept-Dec 2017:  0
User:  168882529 . \# tweets found Sept-Dec 2017:  17
User:  1932816230 . \# tweets found Sept-Dec 2017:  20
User:  241642996 . \# tweets found Sept-Dec 2017:  0
User:  857131219 . \# tweets found Sept-Dec 2017:  18
User:  29029802 . \# tweets found Sept-Dec 2017:  0
User:  189656687 . \# tweets found Sept-Dec 2017:  20
User:  826278546096136193 . \# tweets found Sept-Dec 2017:  20
User:  20193551 . \# tweets found Sept-Dec 2017:  20
User:  1901928318 . \# tweets found Sept-Dec 2017:  0
User:  14404094 . \# tweets found Sept-Dec 2017:  20
User:  46001628 . \# tw

KeyboardInterrupt: 

In [101]:
tweets


[]

In [ ]:

import tweepy
import datetime
import xlsxwriter
import sys

# credentials from https://apps.twitter.com/
consumerKey = "PQSsqSxGft69D53XGknXya8cL"
consumerSecret = "dlh6EFLADanVNvdl9Q9Lz9Iv3yQQyHY9vxKTS9BN2skflGd45d"
accessToken = "1187382763432771584-rwZbZc9lcOcaVdgxcIgqqCPqoQMo3I"
accessTokenSecret = "nvnIFiZD9L1u6tZwntl24BtsLhKmkBntJ4dpko82fYKg8"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)

api = tweepy.API(auth)

username = 756551449149046785
startDate = datetime.datetime(2017, 8, 1, 0, 0, 0)
endDate =   datetime.datetime(2017, 10, 1, 0, 0, 0)

tweets = []
max_guess =  905274232590004225 #902000000000000000
tmpTweets = api.user_timeline(username, max_id = max_guess)


In [69]:
tmpTweets

[]

In [71]:
for tweet in tmpTweets:
    if tweet.created_at < endDate and tweet.created_at > startDate:
        tweets.append(tweet)

In [75]:
tweets[2].text


'@The_Chief1114 big fan of your stream. I tuned in for the first time in days only to have a perma ban in chat. talk to ur mods please? idk'

In [ ]:
while (tmpTweets[-1].created_at > startDate):
    print("Last Tweet @", tmpTweets[-1].created_at, " - fetching some more")
    tmpTweets = api.user_timeline(username, max_id = tmpTweets[-1].id)
    for tweet in tmpTweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            tweets.append(tweet)

workbook = xlsxwriter.Workbook(username + ".xlsx")
worksheet = workbook.add_worksheet()
row = 0
for tweet in tweets:
    worksheet.write_string(row, 0, str(tweet.id))
    worksheet.write_string(row, 1, str(tweet.created_at))
    worksheet.write(row, 2, tweet.text)
    worksheet.write_string(row, 3, str(tweet.in_reply_to_status_id))
    row += 1

workbook.close()
print("Excel file ready")